## Interact with NGSI-LD context broker usign the data model

Uses orion-ld broker

copy updated context to kubernetes using:

In [1]:
from ngsildclient import Entity, Client

In [7]:
host = "localhost"
client = Client(hostname=host, port=8080, port_temporal=8080)
ctx = "http://nginx:8000/context-ngsi.jsonld"
#ctx = "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"

Java-Spring based Context Broker detected. Try to enable info endpoint.

Connected to Context Broker at localhost:8080 | tenant=N/A | vendor=None | version=N/A

In [8]:
# Create a cow
import datetime
import numpy as np
start = datetime.datetime.fromisoformat("2023-01-01T00:00:00")
cow = Entity("AnimalDailyObserved", "cow4", ctx = ctx)
cow.prop("methaneyield", np.random.randint(30, 60), observedat=start)
cow.prop("net_energy_requirement", np.random.randint(700, 1000), observedat=start)
#cow.tprop("observedAt", start) # This gives errors from orion
cow["observedAt"] = start.isoformat() + "Z" # This works
#cow.obs() This one works for some reason
cow.pprint()
#client.create(cow)

{
  "id": "urn:ngsi-ld:AnimalDailyObserved:cow4",
  "type": "AnimalDailyObserved",
  "@context": "http://nginx:8000/context-ngsi.jsonld",
  "methaneyield": {
    "type": "Property",
    "value": 38,
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "net_energy_requirement": {
    "type": "Property",
    "value": 808,
    "observedAt": "2023-01-01T00:00:00Z"
  },
  "observedAt": "2023-01-01T00:00:00Z"
}

In [12]:
%%time
# Create and update the values
for i in range(10):
    cow = Entity("AnimalDailyObserved", f"ttcow{i}", ctx = ctx)
    cow.prop("methaneyield", np.random.randint(30, 60))
    cow.prop("net_energy_requirement", np.random.randint(700, 1000))
    #cow.tprop("observedAt", start) # This gives errors from orion
    #cow["observedAt"] = start.isoformat() + "Z" # This works
    #cow.obs() This one works for some reason
    #cow.pprint()
    #client.create(cow)
    for t in range(5):
        tn = start + datetime.timedelta(days=t)
        cow.prop("methaneyield", np.random.randint(30, 60), observedat=tn)
        cow.prop("net_energy_requirement", np.random.randint(700, 1000), observedat=tn)
        #cow.prop("observedAt", start.isoformat())
        #cow["observedAt"] =tn.isoformat() + "Z"
        #cow.pprint()
        client.update(cow)

CPU times: user 530 ms, sys: 15.9 ms, total: 546 ms
Wall time: 16.9 s


In [13]:
animals = client.query("AnimalDailyObserved", ctx=ctx)
client.temporal.get(animals[0])

{'id': 'urn:ngsi-ld:AnimalDailyObserved:ttcow0',
 'type': 'https://smart-data-models.github.io/data-models#AnimalDailyObserved',
 'https://smart-data-models.github.io/data-models#methaneyield': {'type': 'Property',
  'values': [[35.0, '2023-01-05T00:00:00Z']]},
 'https://smart-data-models.github.io/data-models#net_energy_requirement': {'type': 'Property',
  'values': [[953.0, '2023-01-05T00:00:00Z']]},
 '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context-v1.3.jsonld']}

## Query time series

In [46]:
#client.temporal.get(animals[0])

In [30]:
# See https://ngsi-ld-tutorials.readthedocs.io/en/latest/short-term-history.html

query using requests:

In [19]:
import requests
import datetime

url = 'http://localhost:8080/ngsi-ld/v1/temporal/entities/' 
headers = {
    'Link': f'<{ctx}>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"',
}
params = {
    'type': 'AnimalDailyObserved',
    'lastN': 100,
    'timeproperty': 'observedAt',
    #'timeproperty': 'dateObserved',
    'timerel': 'before',
    'timeAt': f'{datetime.datetime.now().isoformat()}z',
}

response = requests.get(url, headers=headers, params=params)
cowdata = response.json()
cowdata[1]

{'id': 'urn:ngsi-ld:AnimalDailyObserved:ttcow1',
 'type': 'AnimalDailyObserved',
 'methaneyield': [{'type': 'Property',
   'value': 46,
   'instanceId': 'urn:ngsi-ld:Instance:630cef56-01b5-43b4-81d8-8651513e0ebf',
   'observedAt': '2023-01-05T00:00:00Z'}],
 'net_energy_requirement': [{'type': 'Property',
   'value': 959,
   'instanceId': 'urn:ngsi-ld:Instance:cd3653f0-b371-44bc-8c43-6367130a588f',
   'observedAt': '2023-01-05T00:00:00Z'}]}

In [20]:
%%timeit
response = requests.get(url, headers=headers, params=params)

249 ms ± 78.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
#while True:
#    response = requests.get(url, headers=headers, params=params)